# 0. Imports

In [ ]:
import numpy as np
from utils.general_utils import load_data, load_files, study_best_params
from utils.nn_utils import train_model_nn
from sklearn.model_selection import train_test_split
import torch

# 1. Loads

## 1.1. Loading train and test filepaths and labels

In [10]:
processed_data = load_files('../../data/set_pairs/processed_data')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_val_set_1, train_val_set_2 = load_data(processed_data,  '../../data/full_dataset', 'train')
test_set_1, test_set_2 = load_data(processed_data, '../../data/full_dataset', 'test')
train_val_labels = torch.Tensor(processed_data['train_labels']).reshape(-1, 1)
test_labels = torch.Tensor(processed_data['test_labels']).reshape(-1, 1)


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


In [3]:

"""import pickle
names_to_save = ['train_val_set_1', 'train_val_set_2', 'test_set_1', 'test_set_2', 'train_val_labels', 'test_labels']
vars_to_save = [train_val_set_1, train_val_set_2, test_set_1, test_set_2, train_val_labels, test_labels]
for i, name in enumerate(names_to_save):
    with open(f'../../data/to_colab/{name}', 'wb') as f:
        pickle.dump(vars_to_save[i], f)
"""




"import pickle\nnames_to_save = ['train_val_set_1', 'train_val_set_2', 'test_set_1', 'test_set_2', 'train_val_labels', 'test_labels']\nvars_to_save = [train_val_set_1, train_val_set_2, test_set_1, test_set_2, train_val_labels, test_labels]\nfor i, name in enumerate(names_to_save):\n    with open(f'../../data/to_colab/{name}', 'wb') as f:\n        pickle.dump(vars_to_save[i], f)\n"

In [ ]:
search_space = {
    'batch_size': {'suggestion': 'categorical', 'choices': [16, 32, 64, 128], 'listed': None},
    'activation': {'suggestion': 'categorical', 'choices': ["ReLU", "leaky_relu", "GELU", "ELU"], 'listed': None},
    'scheduler': {'suggestion': 'categorical', 'choices': ["linear", "cosine", "plateau"], 'listed': None},
    'num_epochs': {'suggestion': 'int', 'low': 10, 'high': 200, 'log': False, 'listed': None},
    'learning_rate': {'suggestion': 'float', 'low': 1e-6, 'high': 1e-2, 'log': True, 'listed': None},
    'num_convs': {'suggestion': 'int', 'low': 3, 'high': 5, 'log': False, 'listed': None},
    'convs_per_block': {'suggestion': 'int', 'low': 1, 'high': 4, 'log': False, 'listed': None},
    'channels': {'suggestion': 'int', 'low': 4, 'high': 512, 'log': True, 'listed': 'num_convs'},
    'kernel_size': {'suggestion': 'categorical', 'choices': [3, 5, 7, 9], 'listed': 'num_convs'}
}

params_initial = {
    'batch_size': 128,
    'activation': 'ReLU',
    'scheduler': "cosine",
    'num_epochs': 20,
    'learning_rate': 3e-6,
    'num_convs': 5,
    'channels': [32, 64, 128, 128, 256],
    'kernel_size': [11, 6, 4, 4, 3]
}

params_trash = {
    'batch_size': 64,
    'activation': 'ReLU',
    'scheduler': "cosine",
    'num_epochs': 40,
    'learning_rate': 1e-5,
    'num_convs': 3,
    'channels': [16, 32, 256],
    'kernel_size': [13, 7, 5]
}

In [5]:
train_1, val_1, train_2, val_2, train_out, val_out = train_test_split(train_val_set_1, train_val_set_2, train_val_labels, test_size=0.2, random_state=42)


In [6]:
val_loss, model, scaler = train_model_nn(train_1, train_2, train_out, val_1, val_2, val_out, params_trash, device, check_val=True)
print("wtg")


epoch: 0, loss: 0.7098206119103865, val_loss: 0.7188652753829956
epoch: 1, loss: 0.7083708654750477, val_loss: 0.7112004160881042
epoch: 2, loss: 0.7068823467601428, val_loss: 0.7104777097702026
epoch: 3, loss: 0.701743180101568, val_loss: 0.6924492120742798
epoch: 4, loss: 0.6975157087499446, val_loss: 0.7028732895851135
wtg


In [7]:

models, scalers, scores = study_best_params(train_val_set_1,
                                            train_val_set_2,
                                            train_val_labels,
                                            search_space,
                                            device,
                                            k=5,
                                            print_message=False,
                                            num_iters=50)

[I 2025-06-05 14:13:33,602] A new study created in memory with name: no-name-6b692dc6-27c1-442b-b93d-5e261a35856f



Parameters: {'batch_size': 32, 'activation': 'GELU', 'scheduler': 'linear', 'num_epochs': 107, 'learning_rate': 0.0005700606222328687, 'num_convs': 3, 'channels': [175, 371, 49], 'kernel_size': [7, 7, 5]}




[W 2025-06-05 14:13:33,966] Trial 0 failed with parameters: {'batch_size': 32, 'activation': 'GELU', 'scheduler': 'linear', 'num_epochs': 107, 'learning_rate': 0.0005700606222328687, 'num_convs': 3, 'channels_0': 175, 'channels_1': 371, 'channels_2': 49, 'kernel_size_0': 7, 'kernel_size_1': 7, 'kernel_size_2': 5} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 642.00 MiB. GPU 0 has a total capacity of 10.91 GiB of which 262.75 MiB is free. Including non-PyTorch memory, this process has 10.22 GiB memory in use. Of the allocated memory 9.56 GiB is allocated by PyTorch, and 505.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)').
Traceback (most recent call last):
  File "/home/ilan/PycharmProjects/Siamese_roy/.ve

OutOfMemoryError: CUDA out of memory. Tried to allocate 642.00 MiB. GPU 0 has a total capacity of 10.91 GiB of which 262.75 MiB is free. Including non-PyTorch memory, this process has 10.22 GiB memory in use. Of the allocated memory 9.56 GiB is allocated by PyTorch, and 505.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
torch.cuda.empty_cache()
print(f"allocated: {torch.cuda.memory_allocated() / 10**6} MB")
print(f"cached: {torch.cuda.memory_reserved() / 10**6} MB")


AttributeError: 'torch.device' object has no attribute 'empty_cache'